In [20]:
from timeit import timeit as time

# Make TFIDF

In [21]:
import pdb 
import os
from gensim import corpora, models, similarities, matutils, interfaces, utils
from nltk.corpus import stopwords
import numpy as np
import logging
import multiprocessing

pool_size = multiprocessing.cpu_count() - 1

STOP_WORDS = stopwords.words('english')
DATA_PATH = (u'./pmc_data/pmc_text_files/')
SAVE_LOCATION = './pmc_models_serialized/'
load_corpus_times = []

class PubmedCorpus(object):
    def __init__(self, data_folder=DATA_PATH):
        self.data_folder = data_folder
        self.dictionary = corpora.Dictionary()
        self.document_file_names = []
        
    def __iter__(self):
        pool = multiprocessing.Pool(pool_size)        
        for file_chunk in utils.chunkize(self.file_path_iter(), chunksize=100, maxsize=2):
            docs = pool.imap(tokenized_from_file, file_chunk)
            for doc_tokenized in docs:
                yield self.dictionary.doc2bow(doc_tokenized)
        pool.terminate()                      
### TODO clean up redundancy    

    def load_corpus(self):
        pool = multiprocessing.Pool(pool_size)
        for file_chunk in utils.chunkize(self.file_path_iter(), chunksize=100 , maxsize=2):
            results = pool.imap(tokenized_from_file, file_chunk)
#             results = [i for i in results]
            self.dictionary.add_documents(results)
            self.document_file_names += [file_path for file_path in file_chunk]
        pool.terminate()            

    def file_path_iter(self):
        for root, dirs, files in os.walk(self.data_folder):
            for file_name in files:
                file_path = os.path.join(root, file_name)
                yield file_path
                
def tokenized_from_file(file_path):
    with open(file_path, 'rb') as f:
        doc = f.read()
        doc_token_gen = utils.tokenize(doc, lowercase=True)
        doc_tokenized = [i for i in doc_token_gen if i not in STOP_WORDS]
        return doc_tokenized
        
        
def to_unicode_or_bust(
        obj, encoding='utf-8'):
    if isinstance(obj, basestring):
        if not isinstance(obj, unicode):
            obj = unicode(obj, encoding)
    return obj
              
pubmed_corpus = PubmedCorpus()


tfidf = models.tfidfmodel.TfidfModel.load('tfidf')

lsi = models.LsiModel.load('lsi')

Create an index transformer that calculates similarity based on our space
 
test_index1 = similarities.MatrixSimilarity(test_lsi1)

In [22]:
pubmed_corpus.load_corpus()

In [24]:
pubmed_corpus.document_file_names

[u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2007_Apr_11_92(4)_298-303.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2008_Aug_18_93(8)_654-659.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2008_Jul_1_93(7)_566-569.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2008_Jul_1_93(7)_620-625.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2008_May_17_93(5)_373-378.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2008_Oct_2_93(10)_845-850.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2008_Oct_30_93(10)_890-898.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2009_Feb_14_94(2)_110-116.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2009_Feb_19_94(2)_156-160.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2009_Jan_25_94(1)_11-15.txt',
 u'./pmc_data/pmc_text_files/Arch_Dis_Child/Arch_Dis_Child_2009_Jan_9_94(1)_42-46.txt',
 u'./

In [10]:
pubmed_corpus.dictionary.doc2bow(['yo', 'yellow', 'circumference', 'obesity'])

[]

- save pubmed_corpus.dictionary
    - pubmed_corpus.dictionary.save()
- 

##### `tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model
`

In [ ]:
pubmed_tfidf = models.TfidfModel(pubmed_corpus, normalize=True)
pubmed_tfidf.save(os.path.join(SAVE_LOCATION, 'pubmed_tfidf'))

##### Transform incoming vectors

`doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors
[(0, 0.70710678), (1, 0.70710678)]`

##### Transform the entire corpus (lazy, only does so when run)

```corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
...     print(doc)
[(0, 0.57735026918962573), (1, 0.57735026918962573), (2, 0.57735026918962573)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.32448702061385548), (6, 0.44424552527467476), (7, 0.32448702061385548)]
[(2, 0.5710059809418182), (5, 0.41707573620227772), (7, 0.41707573620227772), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.71848116070837686), (8, 0.49182558987264147)]
[(3, 0.62825804686700459), (6, 0.62825804686700459), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.70710678118654746), (10, 0.70710678118654746)]
[(9, 0.50804290089167492), (10, 0.50804290089167492), (11, 0.69554641952003704)]
[(4, 0.62825804686700459), (10, 0.45889394536615247), (11, 0.62825804686700459)]```

In [ ]:
pubmed_corpus_tfidf = pubmed_tfidf[pubmed_corpus]
pubmed_corpus_tfidf.save(os.path.join(SAVE_LOCATION, 'pubmed_corpus_tfidf'))

In [ ]:
pubmed_corpus.dictionary.

In [ ]:
??utils.chunkize

##### LSI

In [ ]:
pubmed_lsi = models.LsiModel(pubmed_corpus_tfidf, 
                             id2word=pubmed_corpus.dictionary, 
                             num_topics=200)
pubmed_corpus_lsi = pubmed_lsi[pubmed_corpus_tfidf]
pubmed_lsi.save(os.path.join(SAVE_LOCATION, 'pubmed_lsi'))

##### look at some of the topics

In [ ]:
pubmed_lsi.print_topics(2)

### Fold in new document

In [ ]:
new_doc = 'hello there overweight circumference pregnancy'.split()

In [ ]:
new_vec = pubmed_corpus.dictionary.doc2bow(new_doc)

In [ ]:
new_vec

In [ ]:
new_tfidf_vec = pubmed_tfidf[new_vec]

In [ ]:
dictionary = corpora.Dictionary()

In [ ]:
dictionary.doc2bow(['hello', 'something'])

In [ ]:
class thing:
    def __init__(self):
        self.array = [(1, 2, 3), (3,3, 3) , (2, 2, 2)]
    
    def __iter__(self):
        for i, j, k in self.array:
            yield i * 2
            
    def load_stuff(self):
        for i in self:
            print i
            
    def again(self):
        for i in self:
            print i

In [ ]:
a = thing()

In [ ]:
a.load_stuff()

In [ ]:
# class PubmedCorpus(corpora.TextCorpus):
    
#     def get_texts(self):
#         directory = os.walk(self.input)
#         for root, dirs, files in directory:
#             for file_name in files:
#                 file_path = os.path.join(root, file_name)
#                 with open(file_path, 'rb') as f:
#                     file_string = to_unicode_or_bust(f.read().lower())
#                     file_tokenized = word_tokenize(file_string)
#                     file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
#                     yield file_no_stops

# class PubmedCorpus(corpora.TextCorpus):
        
#     def get_texts(self):
#         for file_path in self.input:
#             with open(file_path, 'rb') as f:
#                 file_string = to_unicode_or_bust(f.read().lower())
#                 file_tokenized = word_tokenize(file_string)
#                 file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
#                 yield file_no_stops
                    
                
# def load_article_paths():
#     for root, dirs, files in os.walk('./pmc_data/pmc_text_files/'):
#         for name in files:
#             ARTICLE_FILE_PATHS.append(os.path.join(root, name))
#     genre_folders_left = len(GENRE_FOLDERS)
#     completed_genre_folders = 0 
#     for genre_folder in GENRE_FOLDERS:
#         completed_genre_folders += 1
#         genre_folder_path = os.path.join(DATA_PATH, genre_folder)
#         genre_file_list = os.listdir(genre_folder_path)
#         for article_file_title in genre_file_list:
#             article_file_path = os.path.join(genre_folder_path, article_file_title)
#             if os.path.isfile(article_file_path):
#                 ARTICLE_FILE_TITLES.append(article_file_title)
#                 ARTICLE_FILE_PATHS.append(article_file_path)
# #                 with open(article_file_path, 'rb') as f:
# #                     document = f.read()
# #                     ARTICLE_DOCUMENT_LIST.append(document)

#                 print "done with: ", article_file_title
#                 print "progress: ", completed_genre_folders / float(genre_folders_left)
#             else:
#                 sub_article_folder_list = os.listdir(article_file_path)
#                 for sub_article_file_title in sub_article_folder_list:
#                     sub_article_file_path = os.path.join(article_file_path, 
#                             sub_article_file_title)
#                     ARTICLE_FILE_TITLES.append(sub_article_file_title)
#                     ARTICLE_FILE_PATHS.append(sub_article_file_path)
# #                     with open(sub_article_file_path, 'rb') as f:
# #                         document = f.read()
# #                         ARTICLE_DOCUMENT_LIST.append(document)

### Pubmed Corpus from corpora.TextCorpus

In [ ]:
# class PubmedCorpus(corpora.TextCorpus):
#     def __init__(self, input=None, data_folder=DATA_PATH):
#         super(TextCorpus, self).__init__()
#         self.input = None
#         self.data_folder = os.walk(data_folder)
#         self.dictionary = Dictionary()
#         self.metadata = False        
#         if data_folder is not None:
#             self.dictionary.add_documents
    
#     def __iter__(self):
        
        
#     def get_texts(self, file_name):
#         with open(file_name, 'rb') as doc_file:
#             doc = doc_file.read()
#             doc_tokenized = utils.tokenize(doc, lowercase=True)
#             doc_tokenized = [word for word in doc_tokenized if word not in STOP_WORDS]
#             yield doc_tokenized

                    
# #         for doc_file in self.input:
# #             with open(doc_file, 'rb') as doc:
# #                 doc_text = doc.read()
# #                 tokenized_text = [token for token in utils.tokenize(doc_text, 
# #                                                                     lowercase=True)]
# #                 tokenized_text = [word for word in tokenized_text if word not in STOP_WORDS]
# #                 self.dictionary.add_documents([tokenized_text])
# #                 yield self.dictionary.doc2bow(tokenized_text, allow_update=False)
                
        
        
        
        
# #         directory = os.walk(self.input)
# #         for root, dirs, files in directory:
# #             for file_name in files:
# #                 file_path = os.path.join(root, file_name)
# #                 with open(file_path, 'rb') as f:
# #                     file_string = to_unicode_or_bust(f.read().lower())
# #                     file_tokenized = word_tokenize(file_string)
# #                     file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
# #                     yield file_no_stops